# 2022-09-15

In [3]:
%matplotlib inline

In [4]:
import matplotlib.pyplot as plt

In [5]:
import numpy as np

In [6]:
! pip install tensorboardX

In [7]:
%load_ext tensorboard

In [8]:
from tensorboardX import SummaryWriter

In [9]:
from datetime import datetime

In [10]:
logdir = "logs"

In [11]:
def activation(x):
  return 1.0 / (1.0 + np.exp(-x))

In [12]:
def dactivation(x):
  return np.exp(-x)/(1.0 + np.exp(-x))**2

In [13]:
#  (2,3,1)
class MLP:

  def __init__(self,*args):
    np.random.seed(123)
    self.shape = args
    n = len(args)

    self.layers= []

    self.layers.append(np.ones(self.shape[0]+1))

    for i in range (1,n):
      self.layers.append(np.ones(self.shape[i]))

    self.weights = []
    for i in range(n-1):
      self.weights.append(np.zeros((self.layers[i].size,self.layers[i+1].size)))

    self.dw = [0,]*len(self.weights)

    self.reset()

  def reset(self):
    for i in range(len(self.weights)):
      Z = np.random.random((self.layers[i].size,self.layers[i+1].size))

      self.weights[i][...] = (2*Z-1)*1

  def propagate_forward(self,data):

    self.layers[0][0:-1] = data
    
    for i in range(1,len(self.shape)):
      s_i = np.dot(self.layers[i-1], self.weights[i-1])
      self.layers[i][...] =  activation(s_i)
    return self.layers[-1]
  def propagate_backward(self,target,lrate=0.1):
    deltas = []

    derror = -(target-self.layers[-1])
    
    s_last = np.dot(self.layers[-2],self.weights[-1])
    delta_last = derror *  dactivation(s_last)
    deltas.append(delta_last)
    
    for i in range(len(self.shape)-2,0,-1):
      s_i = np.dot(self.layers[i-1],self.weights[i-1])
      delta_i = np.dot(deltas[0],self.weights[i].T)*dactivation(s_i)
      deltas.insert(0,delta_i)

    for i in range(len(self.weights)):
      layer = np.atleast_2d(self.layers[i])
      delta = np.atleast_2d(deltas[i])

      dw = -lrate*np.dot(layer.T,delta)

      self.weights[i]+=dw

      self.dw[i]= dw
    error = (target -  self.layers[-1])**2
    return error.sum()

from sklearn import preprocessing
def learn(network,X,Y,valid_split,test_split,write,epochs=20,lrate=0.1):

  nb_samples = len (Y)
  X_train = X [0:int(nb_samples*(1-valid_split-test_split))]
  Y_train = Y [0:int(nb_samples*(1-valid_split-test_split))]
  X_valid = X [int(nb_samples*(1-valid_split-test_split)):int(nb_samples*(1-test_split))]
  Y_valid = Y [int(nb_samples*(1-valid_split-test_split)):int(nb_samples*(1-test_split))]
  X_test = X  [int(nb_samples*(1-test_split)):]
  Y_test = Y  [int(nb_samples*(1-test_split)):]

  scaler = preprocessing.StandardScaler().fit(X_train)
  X_train = scaler.transform(X_train)
  X_valid = scaler.transform(X_valid)
  X_test = scaler.transform(X_test)

  randperm = np.random.permutation(len(X_train))
  X_train,Y_train = X_train[randperm],Y_train[randperm]

  for i in range (epochs):
    train_err = 0 
    for k in range (X_train.shape[0]):
      network.propagate_forward(X_train[k])
      train_err +=network.propagate_backward(Y_train[k],lrate)
    train_err /=X_train.shape[0]

    valid_err = 0
    o_valid = np.zeros(X_valid.shape[0])
    for k in range(X_valid.shape[0]):
      o_valid[k] = network.propagate_forward(X_valid[k])
      valid_err += (o_valid[k]-Y_valid[k])**2
    valid_err /=X_valid.shape[0]

    write.add_scalar('train',scalar_value=train_err,global_step=i)
    write.add_scalar('validation',scalar_value=valid_err,global_step=i)
    print("{} epoch, train_err: {}, valid_err: {}".format(i,train_err, valid_err))

  print("TESZT")
  test_err = 0
  o_test = np.zeros(X_test.shape[0])
  for k in range(X_test.shape[0]):
    o_test[k] = network.propagate_forward(X_test[k])
    test_err += (o_test[k]-Y_test[k])**2
  test_err /= X_test.shape[0]
  print(test_err)               

In [14]:
network = MLP (2,10,1)

In [15]:
nb_samples=1000
X = np.zeros((nb_samples,2))
Y = np.zeros(nb_samples)
for i in range(0,nb_samples,4):
    noise = np.random.normal(0,1,8)
    X[i], Y[i] = (-2+noise[0],-2+noise[1]), 0
    X[i+1], Y[i+1] = (2+noise[2],-2+noise[3]), 1
    X[i+2], Y[i+2] = (-2+noise[4],2+noise[5]), 1
    X[i+3], Y[i+3] = (2+noise[6],2+noise[7]), 0

In [16]:
# a pontos időt lekérdezzük majd string-re alakítjuk
now = datetime.now()
date_time = now.strftime("%Y%m%d_%H-%M-%S")
writer = SummaryWriter(logdir+"/"+date_time, flush_secs=1)


# Tanítás/Tesztelés indítása
network.reset()
valid_split = 0.2; test_split = 0.1
learn(network, X, Y, valid_split, test_split, writer, 100)

0 epoch, train_err: 0.24614314439761184, valid_err: 0.23784604654950872
1 epoch, train_err: 0.2297025824522868, valid_err: 0.2211767844090273
2 epoch, train_err: 0.20695727395811084, valid_err: 0.19340898826237177
3 epoch, train_err: 0.17415096809613942, valid_err: 0.15847635538727112
4 epoch, train_err: 0.13874775481191173, valid_err: 0.12667624882747902
5 epoch, train_err: 0.10981930239183453, valid_err: 0.10346711821683872
6 epoch, train_err: 0.08970630277330417, valid_err: 0.08786103487832059
7 epoch, train_err: 0.0763555883214336, valid_err: 0.07733361232650038
8 epoch, train_err: 0.06734767241493564, valid_err: 0.06998772032137736
9 epoch, train_err: 0.061033865171153, valid_err: 0.06465801307469875
10 epoch, train_err: 0.05642294582752031, valid_err: 0.06064978918367313
11 epoch, train_err: 0.05292709294356913, valid_err: 0.057540681448743865
12 epoch, train_err: 0.05018970754043007, valid_err: 0.05506522152980132
13 epoch, train_err: 0.04798708867732972, valid_err: 0.0530506000

In [17]:
%tensorboard --logdir logs